<a href="https://colab.research.google.com/github/bluebird702/study/blob/main/reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# google python sdk 설치
!pip install -q -U langchain langchain-google-genai langchain-community cache-pandas

In [2]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
REDDIT_CLIENT_ID=userdata.get('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET=userdata.get('REDDIT_CLIENT_SECRET')

In [35]:
import requests
import pandas as pd
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage
from langchain_community.document_loaders import RedditPostsLoader
import praw

def get_posts(limit=10):
  reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent="extractor by u/Master_Ocelot8179",
  )

  subreddit = reddit.subreddit("wallstreetbets")
  hot_posts = subreddit.hot(limit=10)
  # for submission in subreddit.hot(limit=10):
  #   print(submission.title)
  #   # Output: the submission's title
  #   print(submission.score)
  #   # Output: the submission's score
  #   print(submission.id)
  #   # Output: the submission's ID
  #   print(submission.url)
  #   print(submission.num_comments)
  #   print(submission.selftext)
  #   print("\n\n")
  return hot_posts

def get_sentiment(llm, text):
  prompt = HumanMessage(content=f"Analyze the sentiment of the following text: '{text}'. Return 'positive', 'negative', or 'neutral' based on the sentiment analysis.")
  response = llm([prompt])
  sentiment = response.content.strip().lower()

  if 'positive' in sentiment:
      return 'positive'
  elif 'negative' in sentiment:
      return 'negative'
  else:
      return 'neutral'

def get_keyword(llm, text):
  prompt = HumanMessage(content=f"Extract just 5 keywords that matter from the given text: '{text}'. Return list them separated by commas.")
  response = llm([prompt])
  return response.content.strip().lower()

def process_post(llm, submission):
  text = submission.selftext
  sentiment = get_sentiment(llm, text)
  if sentiment == 'negative':
      keyword = get_keyword(llm, text)
  else:
      keyword = 'N/A'
  post_with_sentiment = {
      'title': submission.title,
      'score': submission.score,
      'sentiment': sentiment,
      'keyword' : keyword,
      'url': submission.url,
      'permalink': submission.permalink,
      'is_self': submission.is_self,
      'selftext': submission.selftext,
      'num_comments': submission.num_comments,
  }
  return post_with_sentiment

def process_posts(llm, submissions):
  return [process_post(llm, submission) for submission in submissions]

# 객체 생성
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY
)

display(get_posts())
#display(process_posts(llm, get_posts()))

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Forbidden: received 403 HTTP response